<a href="https://colab.research.google.com/github/radhakrishnan-omotec/image-classification-nimayseth-repo/blob/main/YOLO11_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training YOLO11 Object Detection and Image Organization on a Custom Dataset

This approach involves training YOLO11 on a custom dataset to detect objects, and then leveraging clustering techniques to organize and group images based on visual similarities among detected objects. This aligns with goals from the research paper for improved digital image categorization and retrieval.

###Requirements

    Roboflow API Key: Required for dataset import.

    GPU Access: Confirm GPU access for faster training
    (Edit -> Notebook settings -> Hardware accelerator -> GPU).

In [ ]:
!nvidia-smi


###Setup and Installation

Define Constants:

In [ ]:
import os
HOME = os.getcwd()
print(HOME)


Install YOLO11:

In [ ]:
# Install necessary libraries
!pip install ultralytics supervision roboflow
import ultralytics
ultralytics.checks()


##Initial Inference and Clustering
###Step 1: Inference with Pre-trained Model

Use the YOLO11 model pre-trained on COCO to test the object detection:

In [ ]:
from ultralytics import YOLO
from PIL import Image
import requests

model = YOLO('yolo11n.pt')
image = Image.open(requests.get('https://media.roboflow.com/notebooks/examples/dog.jpeg', stream=True).raw)
result = model.predict(image, conf=0.25)[0]


###Step 2: Extract Object Data and Cluster Images

After detection, use feature vectors from the detected objects to group and categorize similar images.
Leveraging supervision here provides access to bounding box and label annotations.

In [ ]:
import supervision as sv

detections = sv.Detections.from_ultralytics(result)
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator(text_color=sv.Color.BLACK)

annotated_image = box_annotator.annotate(image.copy(), detections=detections)
annotated_image = label_annotator.annotate(annotated_image, detections=detections)


###Step 3: Clustering Detected Objects for Image Organization

Extract embeddings for each detected object and apply clustering (e.g., k-means) to group images by similar objects.
This step helps create structured clusters for efficient image retrieval.

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# Dummy feature extraction (replace with actual embedding extraction)
features = np.array([detection.box for detection in detections])  # Placeholder
kmeans = KMeans(n_clusters=5).fit(features)
clusters = kmeans.predict(features)


###Step 4:Custom Dataset Preparation and Training
###Dataset Download via Roboflow

Download a custom dataset compatible with YOLO11 for fine-tuning.

In [ ]:
from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

workspace = rf.workspace("user_workspace")
project = workspace.project("your_project_name")
version = project.version(1)
dataset = version.download("yolov11")


###Step 5:Fine-Tune YOLO11 on the Custom Dataset

In [ ]:
!yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=20 imgsz=640 plots=True


###Step 6:Validation and Visualization

In [ ]:
!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml


####Object Detection with Clustering Visualization

This visualization step categorizes images based on detected object clusters to help with organization.

By training YOLO11 and integrating clustering methods for image organization, this approach facilitates automatic sorting of detected objects into meaningful groups for streamlined large-scale image management.

In [ ]:
from IPython.display import Image as IPyImage, display

latest_folder = max(glob.glob(f'{HOME}/runs/detect/predict*/'), key=os.path.getmtime)
for img_path in glob.glob(f'{latest_folder}/*.jpg')[:3]:
    display(IPyImage(filename=img_path, width=600))


###Final Step: Categorize and Save Images into Folders

After fine-tuning YOLO11 and detecting objects, we’ll implement automatic grouping by saving images into folders based on the detected categories. Here’s how:

###Step 7: Extract and Assign Category Labels to Detected Objects

We’ll start by assigning a primary category label to each image based on the object(s) detected.

In [ ]:
import os
import shutil
from ultralytics import YOLO
from PIL import Image
from sklearn.cluster import KMeans
import numpy as np

# Define the directory to save categorized images
output_dir = os.path.join(HOME, 'categorized_images')
os.makedirs(output_dir, exist_ok=True)

# Load model and inference on custom images
model = YOLO(f'{HOME}/runs/detect/train/weights/best.pt')
image_paths = [f for f in glob.glob(f'{dataset.location}/test/images/*.jpg')]

# Loop through images and detect objects
for img_path in image_paths:
    image = Image.open(img_path)
    result = model.predict(image, conf=0.25)[0]

    # Use the detected category with the highest confidence score as the folder name
    categories = [result.names[int(cls)] for cls in result.boxes.cls]
    primary_category = max(set(categories), key=categories.count)

    # Create a directory for the primary category if it doesn’t exist
    category_folder = os.path.join(output_dir, primary_category)
    os.makedirs(category_folder, exist_ok=True)

    # Save the image to the category folder
    shutil.copy(img_path, os.path.join(category_folder, os.path.basename(img_path)))


###Step 8: Folder Verification and Display

After categorization, verify that images are correctly saved into their respective folders based on the main detected category.

###Explanation


*   Primary Category Selection: For each image, we choose the category with the highest frequency or confidence as the folder label.
*   Folder Creation and Image Saving: The script creates a folder for each unique category and saves images into their corresponding folder.
*   Final Check: The script outputs the contents of each folder, showing a sample of categorized images for verification.



In [ ]:
import os

# List folders and their contents
for folder_name in os.listdir(output_dir):
    folder_path = os.path.join(output_dir, folder_name)
    if os.path.isdir(folder_path):
        print(f"\nCategory: {folder_name} (Total Images: {len(os.listdir(folder_path))})")
        for img_file in os.listdir(folder_path)[:3]:  # Show a few samples from each category
            print(f" - {img_file}")


This approach streamlines image organization by automatically grouping similar images in category-named folders, making retrieval and management of large-scale datasets far more efficient.